<a href="https://colab.research.google.com/github/saksham1211/Marketingai_Internship/blob/master/NewsAnalysis_UsingPytorch_internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv('news.csv', encoding = 'unicode_escape')

In [13]:
data.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [0]:
y = data.NewsType
x= data.drop('NewsType', axis=1)

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.head()

,Article,Date,Heading
2310,MANCHESTER: Moeen Ali took two wickets as Eng...,7/25/2016,Pakistan 161 5 at tea as England move closer t...
1959,"Hertogenbosch, Netherlands: Nicolas Mahut warm...",6/13/2016,Mahut claims third s Hertogenbosch ti
823,strong>ISLAMABAD: Pakistan International Airli...,7/29/2016,PIA signs contact lease three A 330s Sri L
1311,strong>ZURICH: Gianni Infantino vowed on Frida...,2/27/2016,Infantino chosen to lead FIFA into new er
1805,strong>LONDON: England paceman Stuart Broad be...,5/26/2016,Blooming partnership Anderson pleases Stuart B...


In [16]:
x_train.shape

(2153, 3)

In [45]:
!pip install torch<=1.2.0
!pip install torchtext
%matplotlib inline

/bin/bash: =1.2.0: No such file or directory


In [46]:
pip install https://github.com/pytorch/text/archive/master.zip

     | 3.9MB 7.2MB/s
     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for torchtext: filename=torchtext-0.6.0-cp36-none-any.whl size=68047 sha256=2fcf74d869b2e50b20d54feeef29f779cdb0e6a581e23920326f8dc5241196f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-avmszk27/wheels/5a/86/3d/30ae7dfdfeb1748bb11b3da173fb9634141fbb39e9e9847317
Successfully built torchtext
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [1]:
import torch

import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
	os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ag_news_csv.tar.gz: 11.8MB [00:00, 128MB/s]
120000lines [00:09, 12809.01lines/s]
120000lines [00:19, 6262.48lines/s]
7600lines [00:01, 6374.38lines/s]


In [0]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [0]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [0]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
  
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [0]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()
    
    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [7]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 29 seconds
	Loss: 0.0261(train)	|	Acc: 84.8%(train)
	Loss: 0.0001(valid)	|	Acc: 90.9%(valid)
Epoch: 2  | time in 0 minutes, 30 seconds
	Loss: 0.0119(train)	|	Acc: 93.7%(train)
	Loss: 0.0001(valid)	|	Acc: 90.7%(valid)
Epoch: 3  | time in 0 minutes, 30 seconds
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0001(valid)	|	Acc: 91.4%(valid)
Epoch: 4  | time in 0 minutes, 31 seconds
	Loss: 0.0038(train)	|	Acc: 98.1%(train)
	Loss: 0.0001(valid)	|	Acc: 91.4%(valid)
Epoch: 5  | time in 0 minutes, 34 seconds
	Loss: 0.0022(train)	|	Acc: 99.0%(train)
	Loss: 0.0001(valid)	|	Acc: 90.3%(valid)


In [9]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0002(test)	|	Acc: 89.8%(test)


In [19]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = str(data['Article'][0])

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Business news
